In [1]:
# This code is solution to a portfolio optimisation problem
# Description: Here, we select 5 most un-correlated stocks from the top 20 stocks contributing to nifty index.
# Selection method is such that, given a correlation matrix of 20 stocks, we select 5 such that their summation of correlation is minimum.

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import itertools

In [3]:
stock_tickers = ['HDFCBANK.NS', 'RELIANCE.NS', 'ICICIBANK.NS', 'INFY.NS', 'ITC.NS', 'TCS.NS', 'AXISBANK.NS', 'KOTAKBANK.NS', 'HINDUNILVR.NS', 'SBIN.NS', 'BHARTIARTL.NS', 'BAJAJFINSV.NS', 'ASIANPAINT.NS', 'M&M.NS', 'MARUTI.NS', 'HCLTECH.NS', 'SUNPHARMA.NS', 'TITAN.NS', 'LTIM.NS', 'ADANIENT.NS']

In [4]:
stock_data = yf.download(stock_tickers, period='1y')

[*********************100%%**********************]  20 of 20 completed


In [5]:
stock_data

Adj Close                                           \
            ADANIENT.NS ASIANPAINT.NS  AXISBANK.NS BAJAJFINSV.NS   
Date                                                               
2022-11-07  3958.622314   3077.646729   872.060120   1776.182617   
2022-11-09  3995.203857   3060.739014   872.809326   1750.859131   
2022-11-10  3990.256348   3019.733887   842.839905   1698.564087   
2022-11-11  4006.848145   3029.898438   849.832764   1717.444336   
2022-11-14  4020.541260   3027.716797   850.132446   1716.545166   
...                 ...           ...          ...           ...   
2023-10-31  2294.649902   2990.479736   981.849976   1569.550049   
2023-11-01  2217.300049   2929.436279   971.799988   1572.849976   
2023-11-02  2215.300049   2950.249756   982.950012   1575.599976   
2023-11-03  2229.850098   2977.850098   988.849976   1536.150024   
2023-11-06  2246.000000   3029.300049  1009.400024   1564.400024   

                                                                               \
           BHARTIARTL.NS   HCLTECH.NS  HDFCBANK.NS HINDUNILVR.NS ICICIBANK.NS   
Date                                                                            
2022-11-07    818.938232  1002.016357  1491.693481   2493.989990   910.009460   
2022-11-09    815.304932  1008.044800  1491.248535   2470.357666   903.066589   
2022-11-10    821.974304  1007.087830  1507.462524   2475.084229   903.264954   
2022-11-11    823.119080  1043.641602  1592.883057   2464.695801   899.148804   
2022-11-14    818.141907  1051.631470  1596.738770   2420.139160   887.643555   
...                  ...          ...          ...           ...          ...   
2023-10-31    914.400024  1276.000000  1476.500000   2465.914307   915.349976   
2023-11-01    913.450012  1258.949951  1474.500000   2454.250000   914.049988   
2023-11-02    923.849976  1273.400024  1476.750000   2486.800049   919.950012   
2023-11-03    930.750000  1266.250000  1483.750000   2509.250000   933.799988   
2023-11-06    939.200012  1269.699951  1494.500000   2497.399902   945.450012   

                         ...    Volume                                \
                INFY.NS  ...    ITC.NS KOTAKBANK.NS LTIM.NS   M&M.NS   
Date                     ...                                           
2022-11-07  1448.957153  ...  10050303      2624601  161003  2599224   
2022-11-09  1452.561646  ...  20611075      2322634  222628  2459869   
2022-11-10  1443.670410  ...  19327625      2510124  307460  2792546   
2022-11-11  1509.176147  ...  10950902      3424030  771362  5308986   
2022-11-14  1523.786377  ...  17639889      2142267  664125  3794391   
...                 ...  ...       ...          ...     ...      ...   
2023-10-31  1368.400024  ...   8620154      4839336  259121  2555737   
2023-11-01  1354.150024  ...   7990245      2528105  323448  2627536   
2023-11-02  1370.150024  ...   8152031      2550558  312381  2091122   
2023-11-03  1389.050049  ...  13802600      1558958  324952  2456723   
2023-11-06  1403.300049  ...   7306618      1186483  155126  1271659   

                                                                           
           MARUTI.NS RELIANCE.NS   SBIN.NS SUNPHARMA.NS   TCS.NS TITAN.NS  
Date                                                                       
2022-11-07    469017     5440967  44265903      2853521  1474498  1825529  
2022-11-09    617256     5193814  18939854      2268760  1162267  1184702  
2022-11-10    445700     2753420  11618405      2419991  1573092  1198585  
2022-11-11    461074     5681124  15525267      2390487  3265394   833729  
2022-11-14    282822     4173395  12957268      1764788  1342074   565428  
...              ...         ...       ...          ...      ...      ...  
2023-10-31    439996     6404219  13039696      6475786   999811  1235478  
2023-11-01    699116     5154595  13575575      3973264  1563475   756820  
2023-11-02    569194     5281052  11148516      3454692  1245384   774180  
2023-

In [6]:
stock_closing_prices = stock_data['Adj Close']

In [7]:
correlation_matrix = stock_closing_prices.corr()

In [8]:
correlation_matrix

,ADANIENT.NS,ASIANPAINT.NS,AXISBANK.NS,BAJAJFINSV.NS,BHARTIARTL.NS,HCLTECH.NS,HDFCBANK.NS,HINDUNILVR.NS,ICICIBANK.NS,INFY.NS,ITC.NS,KOTAKBANK.NS,LTIM.NS,M&M.NS,MARUTI.NS,RELIANCE.NS,SBIN.NS,SUNPHARMA.NS,TCS.NS,TITAN.NS
ADANIENT.NS,1.000000,0.236584,0.163653,0.572137,0.147905,-0.251105,-0.090715,0.274861,0.109392,0.486075,-0.545601,0.337469,-0.205292,-0.076213,-0.145151,0.607832,0.668885,0.038885,-0.142738,-0.118967
ASIANPAINT.NS,0.236584,1.000000,0.732647,0.716308,0.722365,0.433306,0.186923,0.481901,0.869985,-0.130647,0.623846,0.411114,0.575148,0.656470,0.615948,0.597396,0.739498,0.515456,0.301226,0.733277
AXISBANK.NS,0.163653,0.732647,1.000000,0.570703,0.842305,0.718983,-0.143580,0.336811,0.741760,-0.019913,0.570396,-0.007445,0.596457,0.811118,0.796075,0.193708,0.654021,0.709374,0.535714,0.840719
BAJAJFINSV.NS,0.572137,0.716308,0.570703,1.000000,0.756090,0.351362,-0.130867,0.261271,0.650735,0.316985,0.189773,0.361154,0.479039,0.522509,0.554366,0.593887,0.791968,0.484207,0.312390,0.574469
BHARTIARTL.NS,0.147905,0.722365,0.842305,0.756090,1.000000,0.800184,-0.272572,0.111610,0.789298,0.115256,0.595316,0.013521,0.764563,0.841133,0.898752,0.178510,0.607115,0.829435,0.626895,0.911341
HCLTECH.NS,-0.251105,0.433306,0.718983,0.351362,0.800184,1.000000,-0.315198,-0.066813,0.577147,0.072437,0.677955,-0.276237,0.869775,0.834604,0.921058,-0.200601,0.218369,0.775857,0.771358,0.850068
HDFCBANK.NS,-0.090715,0.186923,-0.143580,-0.130867,-0.272572,-0.315198,1.000000,0.415207,0.142968,-0.258712,0.130244,0.516358,-0.213032,-0.125677,-0.279802,0.415292,0.094311,-0.248037,-0.134976,-0.186441
HINDUNILVR.NS,0.274861,0.481901,0.336811,0.261271,0.111610,-0.066813,0.415207,1.000000,0.262732,-0.113023,0.135455,0.428107,-0.060021,0.128223,0.018312,0.595310,0.407495,-0.081478,-0.087683,0.096252
ICICIBANK.NS,0.109392,0.869985,0.741760,0.650735,0.789298,0.577147,0.142968,0.262732,1.000000,-0.176828,0.696355,0.428392,0.665359,0.701415,0.738346,0.406582,0.694861,0.641850,0.420469,0.832715
INFY.NS,0.486075,-0.130647,-0.019913,0.316985,0.115256,0.072437,-0.258712,-0.113023,-0.176828,1.000000,-0.511934,-0.288920,0.090193,0.126465,-0.029136,0.154340,0.153219,0.238970,0.432084,-0.159285


In [9]:
# Generating iterables for looping.

combinations = itertools.combinations(range(20), 5)

In [10]:
min_summation = 20
selected_stocks = []

In [11]:
# iterating through iterables to get 5 stocks such that their summation of correlation is minimum.

for combo in combinations:
    summation = sum(correlation_matrix.iloc[i, j] for i, j in itertools.combinations(combo, 2))
    if summation < min_summation:
        min_summation = summation
        selected_stocks = combo

In [12]:
# making the list of stocks

selected_stock_tickers = []
for i in selected_stocks:
    selected_stock_tickers.append(correlation_matrix.columns[i])


In [13]:
selected_stock_tickers

['ADANIENT.NS', 'HCLTECH.NS', 'HDFCBANK.NS', 'INFY.NS', 'ITC.NS']

In [14]:
min_summation

-0.6065533614071679